In [ ]:
import cv2
import pytesseract
import pandas as pd
import re
import matplotlib.pyplot as plt
from PIL import Image
import os 
import natsort 


In [ ]:
def extract_informations(text):

    pattern = re.compile(r"([A-ZÉÀÈÙÂÊÎÔÛÄËÏÖÜÇ]{2,}) \(([^)]+)\)\. (.+)?")

    # Initialisation des variables
    data = []
    current_nom = None
    current_prenom = None
    current_sujet = ""                      

    # Découper le texte en lignes
    lines = text.split("\n")

    for line in lines:
        line = line.strip()  # Nettoyer les espaces
        
        match = pattern.match(line)  # Vérifier si c'est une nouvelle entrée "NOM (Prénom)"

        if match:
            # Si un précédent sujet existait, l'ajouter avant d'en traiter un nouveau
            if current_nom is not None:
                data.append([current_nom, current_prenom, current_sujet.strip()])

            # Extraire le nom, prénom, et début du sujet
            current_nom = match.group(1)
            current_prenom = match.group(2)
            current_sujet = match.group(3) if match.group(3) else ""
        
        else:
            # Si la ligne ne contient pas un nouveau nom, elle fait partie du sujet précédent
            if current_nom is not None:
                current_sujet += " " + line  # Ajouter la ligne au sujet actuel

    # Ajouter la dernière thèse au DataFrame
    if current_nom is not None:
        data.append([current_nom, current_prenom, current_sujet.strip()])

    # Création du DataFrame
    df_cleaned = pd.DataFrame(data, columns=["Nom", "Prénom", "Sujet"])
    df_cleaned['Sujet'] = df_cleaned['Sujet'].str.replace('- ', '', regex=False)
    df_cleaned['Sujet'] = df_cleaned['Sujet'].str.replace(r'\d+', '', regex=True)

    return df_cleaned

In [14]:
def process_image(image) : 
    # Convertir en niveaux de gris
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    gray = clahe.apply(gray)

    # Récupérer les dimensions
    (h, w) = gray.shape

    # Scinder en deux parties
    left_part = gray[:, :w//2 - 5]   # Partie gauche
    right_part = gray[:,  5 + w//2:]  # Partie droite

    left_text = pytesseract.image_to_string(left_part, lang = 'fra')
    right_text = pytesseract.image_to_string(right_part, lang = 'fra')

    df_left = extract_informations(left_text)
    df_right = extract_informations(right_text)

    df_final = pd.concat([df_left, df_right], ignore_index=True)  # ignore_index évite la duplication des index
    return df_final


In [17]:
def process_folder(input_dir):
    # Generate a timestamped folder 
    df = pd.DataFrame()
    
    
    for image_file in natsort.natsorted(os.listdir(input_dir)):
        image_path = os.path.join(input_dir, image_file)
        image = cv2.imread(image_path)
        print(image_path)
        if image_file.lower().endswith('.png'):  
            print(f"Processing {image_path}...")
            df = pd.concat((df, process_image(image)), ignore_index= True)

    return df

current_dir = os.getcwd()

input_dir = os.path.join(current_dir, '..', 'data', 'pdfs_en_images_png', '1870')  # Relative path to 'lib/data/pdfs_en_images_png'



In [18]:
pd.set_option("display.max_rows", None)      # Show all rows
pd.set_option("display.max_columns", None)   # Show all columns
pd.set_option("display.width", 0)            # Auto-detect width
pd.set_option("display.max_colwidth", None)  # Don't truncate column content

process_folder(input_dir)


/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_1.png
Processing /Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_1.png...
/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_2.png
Processing /Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_2.png...
/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_3.png
Processing /Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_3.png...
/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_4.png
Processing /Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_4.png...
/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_5.png
Processing /Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_5.png...
/Users/georges/Desktop/EPFL/PDB/lib/code/../data/pdfs_en_images_png/1870/page_6.png
Proces

,Nom,Prénom,Sujet
0,ABADIE,Jean-Marie-Charles,"Étude sur la myopie stationnaire et progressive, 240"
1,ANDRIEU,Alphonse,"De l'anthrax, de ses complications et de son traitement, 67"
2,ARCHER,Gabriel,"Étude sur les abcès odontopathiques compliqués de dénudation de l'os maxillaire inférieur, 80 ARNAUDET De la transfusion du sang. 23"
3,AUBE,Paul,Quelques considérations sur les amputations bio-tarsiennes. Relation d'un procédé nouveau. 164
4,BADIN,Vital,Contnsious de la portion. périnéale de l'urèthre chez l’homme. 13
5,BALBAUD,Léonce,Étude sur l'empoisonnement par les moules et autres counillages. au
6,BALLUE,Pierre-Gaston,Des erreurs possibles dans le diagnostic des her nies. 216
7,BARBANCEYS,E.,Étude sur la congulation da sang dans les veines. 144
8,BARBIN,Gustave,De l'influence de diverses maladies et ea particulier de La syphilis sur la grossesse. 2
9,BÉHIER,Augustin,Du bruit de pot
